## extracting from recon3d

In [19]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv
from mass.example_data import create_example_model

MASSpy version: 0.1.6


In [20]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
glycolysis = create_example_model("SB2_Glycolysis")



In [21]:
from cobra_dict import metabolite_to_dict
from cobra_dict import reaction_to_dict 

In [28]:
reaction_list=[]
metabolite_list=[]
for rxn in glycolysis.reactions:

    reaction_list.append(rxn.id)
    print(rxn.id)
    for met in rxn.metabolites:
        if met.id not in metabolite_list:
            metabolite_list.append(met.id)
print("The number of reactions are " + str(len(reaction_list)) +" and the number of metabolites are "+ str(len(metabolite_list)))

HEX1
PGI
PFK
FBA
TPI
GAPD
PGK
PGM
ENO
PYK
LDH_L
DM_amp_c
ADK1
SK_pyr_c
SK_lac__L_c
ATPM
DM_nadh
SK_glc__D_c
SK_amp_c
SK_h_c
SK_h2o_c
The number of reactions are 21 and the number of metabolites are 20


In [29]:
met_df=pd.DataFrame()
for met in metabolite_list:
    rbc_met= glycolysis.metabolites.get_by_id(met)
    m=metabolite_to_dict(rbc_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=pd.concat([met_df, df_2])

met_df=met_df.set_index('id')
met_df

,name,charge,formula,compartment
id,,,,
adp_c,ADP,-3,C10H12N5O10P2,c
atp_c,ATP,-4,C10H12N5O13P3,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
glc__D_c,D-Glucose,0,C6H12O6,c
h_c,H+,1,H,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
fdp_c,"D-Fructose 1,6-bisphosphate",-4,C6H10O12P2,c
dhap_c,Dihydroxyacetone phosphate,-2,C3H5O6P,c
g3p_c,Glyceraldehyde 3-phosphate,-2,C3H5O6P,c


In [30]:
csv_met = os.path.join(data_dir,"rbc_met_gluco.csv")
met_df.to_csv(csv_met)

In [31]:
rxn_df=pd.DataFrame()
for rxn in reaction_list:
    rbc_rxn= glycolysis.reactions.get_by_id(rxn)
    r=reaction_to_dict(rbc_rxn)
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df=pd.concat([rxn_df,df])

rxn_df=rxn_df.set_index('id')
rxn_df


,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,annotation
id,,,,,,,
HEX1,Hexokinase (D-glucose:ATP),"{'adp_c': 1, 'atp_c': -1, 'g6p_c': 1, 'glc__D_...",-1000.0,1000.0,,Glycolysis,NaN
PGI,Glucose-6-phosphate isomerase,"{'f6p_c': 1, 'g6p_c': -1}",-1000.0,1000.0,,Glycolysis,NaN
PFK,Phosphofructokinase,"{'adp_c': 1, 'atp_c': -1, 'f6p_c': -1, 'fdp_c'...",-1000.0,1000.0,,Glycolysis,NaN
FBA,Fructose-bisphosphate aldolase,"{'dhap_c': 1, 'fdp_c': -1, 'g3p_c': 1}",-1000.0,1000.0,,Glycolysis,NaN
TPI,Triose-phosphate isomerase,"{'dhap_c': -1, 'g3p_c': 1}",-1000.0,1000.0,,Glycolysis,NaN
GAPD,Glyceraldehyde-3-phosphate dehydrogenase,"{'_13dpg_c': 1, 'g3p_c': -1, 'h_c': 1, 'nad_c'...",-1000.0,1000.0,,Glycolysis,NaN
PGK,Phosphoglycerate kinase,"{'_13dpg_c': -1, '_3pg_c': 1, 'adp_c': -1, 'at...",-1000.0,1000.0,,Glycolysis,NaN
PGM,Phosphoglycerate mutase,"{'_2pg_c': 1, '_3pg_c': -1}",-1000.0,1000.0,,Glycolysis,NaN
ENO,Enolase,"{'_2pg_c': -1, 'h2o_c': 1, 'pep_c': 1}",-1000.0,1000.0,,Glycolysis,NaN


In [32]:
csv_rxn = os.path.join(data_dir,"rbc_rxn_glyco.csv")
rxn_df.to_csv(csv_rxn)

In [33]:
json_rxn = os.path.join(data_dir,"rbc_rxn_glyco.json")
rxn_df.to_json(json_rxn
,orient="index"
)   